# Workshop: Multi-Agent LLMs and Knowledge Graphs for Monitoring and Sustaining Software Systems

## Create Environment and Install libraries

In [1]:
#!pip install python=3.10.18

import sys 
print(sys.version)
print(sys.executable)
!echo $(python --version)
!pip3 install neo4j
!pip3 install -U "autogen-agentchat" "autogen-ext[openai]" "autogen-ext[ollama]"
!pip3 install matplotlib
!pip3 install networkx
!pip3 install pydriller

3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
C:\Users\simeont\AppData\Local\Programs\Python\Python311\python.exe
$(python --version)



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: C:\Users\simeont\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: C:\Users\simeont\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: C:\Users\simeont\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: C:\Users\simeont\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: C:\Users\simeont\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


## Docker Installation 

In [ ]:
# in bash: 
# docker-compose up -d

## Import libaries

In [2]:
import configparser
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime
from neo4j import GraphDatabase
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

##  Prepare the model (API key and configuration)

In [ ]:
config = configparser.ConfigParser(allow_no_value = True)
config.read('openaiapi.ini')
openai_api_key = config.get('openai', 'OPENAI_API_KEY')

In [ ]:
openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=openai_api_key,
    max_tokens = None,
    temperature = None,
    seed = None,
    top_p = None,
    parallel_tool_calls=False  # Disable for Swarms
)


## Setup Neo4J Database 

In [ ]:
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

# First check for connectivity:
driver.verify_connectivity()

In [ ]:
async def query_neo4j(query: str) -> str:
    """Run a Cypher query against Neo4j and return results. Returns String representation."""
    try:
        with driver.session() as session:
            result = session.run(query)
            records = [record.data() for record in result]
        return repr(records)  
    except Exception as e:
        return repr({"error": str(e)})

In [ ]:

# Second check for connectivity (run actual query):
await query_neo4j("RETURN 'Checking connectivity with Neo4j!' AS msg")

## Agent Setup

In [ ]:
current_date = datetime.now()
current_date

In [ ]:
graph_agent = AssistantAgent(
    name="graph_operator",
    model_client=openai_model_client,
    tools=[query_neo4j],
    system_message ="""Your name is graph_operator. You generate **Cypher queries** and execute them.
                    You have access to a tool called query_neo4j that you can use to run Cypher queries and optionally retrieve results. 
                    Use the schema below: \
                    Node Types:
                   (1) CodeChange 
                    - Properties: 
                        uid: str
                        timestamp: date
                        change_description: str
                        change_location: List[str]   # files, functions, or classes affected
                   (2) SourceCode 
                      - Properties:
                        uid: str
                        code_description: str
                        lines_of_code: int
                   (3) CommitMessage
                     - Properties:
                        uid: str
                        message_text: str
                        author: str
                        commit_hash: str
                    Relationship Types:
                 (1) (n:CodeChange)-[:]->(m:CodeChange)         # Temporal sequence of changes
                 (2) (n:SourceCode)-[:]->(m:CodeChange)         # Source code elements impacted by the change
                 (3) (n:CommitMessage)-[:]->(m:CodeChange)      # Commit that introduced the change
                    Rules:
                - Only use this schema. Do not invent nodes or relationships.
                - Only one Cypher statement per query is allowed.""",
)

In [ ]:
task = f"""
Date: {current_date}.
Commit: a13f9c7 by Alice Smith
Commit Message: "Refactored statistics module to improve clarity and reduce duplication"

Changes:
- Refactoring performed on functions `calculate_sum` and `summarize_report` in `statistics.py`
- Moved helper function `format_output` from `utils/helpers.py` into `statistics.py` for better cohesion
- Reduced lines of code in `summarize_report` from 120 to 80

Context:
- This refactor was done as a follow-up to commit `98df231` (previous bugfix in statistics module).
- Refactor categorized as 'code cleanup' and 'modularity improvement'.
"""

## Start chat:

In [ ]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.run_stream(task="Find information on AutoGen"):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        graph_agent.run_stream(task=task),
        output_stats=True,  # Enable stats printing.
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

## Visualize Graph Content

In [ ]:
def visualize_query(query):
    with driver.session() as session:
        result = session.run(query)
        G = nx.DiGraph()
        for record in result:
            start = record["a"]
            end = record["b"]
            rel = record["r"].type
            G.add_edge(start["uid"], end["uid"], label=rel)

    if len(G.nodes) == 0:
        print("No results returned. Check your query or database contents.")
        return

    plt.figure(figsize=(8,6))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=2000, font_size=10, arrows=True)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.show()

In [ ]:
visualize_query("MATCH (a)-[r]->(b) RETURN a, r, b LIMIT 20")

In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage
from autogen_ext.models.ollama import OllamaChatCompletionClient
from pydantic import BaseModel


class StructuredOutput(BaseModel):
    technical_debt_type: str
    description: str
    location: str

class StructuredSuggestOutput(BaseModel):
    suggested_fix: str 

class StructuredRefactorOutput(BaseModel):
    description_of_fix: str
    refactored_code: str 

class StructuredTestOutput(BaseModel):
    evaluation: str
    accepted: str

model_client = OllamaChatCompletionClient(model="qwen3:0.6b")
ollama_model_client = OllamaChatCompletionClient(
    model="qwen3:0.6b",
    response_format=StructuredOutput,
    max_tokens = None,
    temperature = None,
    seed = None,
    top_p = None,
    parallel_tool_calls=False  # Disable for Swarms

)

ollama_model_client_suggest = OllamaChatCompletionClient(
    model="qwen3:0.6b",
    response_format=StructuredSuggestOutput,
    max_tokens = None,
    temperature = None,
    seed = None,
    top_p = None,
    parallel_tool_calls=False  # Disable for Swarms
)

ollama_model_client_refactor = OllamaChatCompletionClient(
    model="qwen3:0.6b",
    response_format=StructuredRefactorOutput,
    max_tokens = None,
    temperature = None,
    seed = None,
    top_p = None,
    parallel_tool_calls=False  # Disable for Swarms
    
)

ollama_model_client_test = OllamaChatCompletionClient(
    model="qwen3:0.6b",
    response_format=StructuredTestOutput,
    max_tokens = None,
    temperature = None,
    seed = None,
    top_p = None,
    parallel_tool_calls=False  # Disable for Swarms
    
)



Creating Agents

In [4]:
## Create Agents

debt_agent = AssistantAgent(
    name="Technical_debt_identifier",
    model_client=ollama_model_client,
    system_message="Identify technical and architectural debt in provided code.",
)

suggestion_agent = AssistantAgent(
    name="Technical_debt_refactoring_suggestor",
    model_client=ollama_model_client_suggest,
    system_message="Suggest refactoring strategies for given technical debt items.",
)

refactoring_agent = AssistantAgent(
    name="Technical_debt_refactoring_agent",
    model_client=ollama_model_client_refactor,
    system_message="Refactor given code based on provided suggestions.",
)

test_agent = AssistantAgent(
    name="Test_agent",
    model_client=ollama_model_client_test,
    system_message="Evaluate the given code. Provide feedback on whether the refactoring was successful or if further changes are needed."
)


In [ ]:
##Code for validating JSON structure in output from 

def validate_json_structure(data: dict) -> None:
    """
    Validate the structure of the JSON data.
    
    Args:
        data (dict): The JSON data to validate
        
    Raises:
        JSONValidationError: If the JSON structure is invalid
    """
    if not isinstance(data, dict):
        raise Exception("Root element must be a dictionary")
        
    for commit_hash, entries in data.items():
        if not isinstance(entries, list):
            raise Exception(f"Entries for commit {commit_hash} must be a list")
            
        for entry in entries:
            if not isinstance(entry, dict):
                raise Exception(f"Each entry in commit {commit_hash} must be a dictionary")

            if 'technicalDebts' in entry and not isinstance(entry['technicalDebts'], list):
                raise Exception(f"technicalDebts in commit {commit_hash} must be a list")

In [ ]:
from pydriller import Repository
def analyze_commits(repo_url, begin_commit, end_commit, debts, debts_file):
    """
    The function will iterate through the commits and fetch the changed content from the previous commit.

    """
    commit_count = 0
    for commit in Repository(repo_url, from_commit=begin_commit, to_commit=end_commit).traverse_commits():
        print("Analyzing commit: %s", commit.hash)
        print("In the repo: %s", repo_url)
        commit_count += 1

        analyze_modifications(commit, debts, debts_file, repo_url)

In [ ]:
def analyze_modifications(commit, debts, debts_file, repo_url):
    """
    The function will go thorugh each commit in the repo and analyze.

    """
    for modification in commit.modified_files:
        if not modification.source_code or not is_source_code(modification.new_path):
            continue

        print("Analyzing file: %s", modification.new_path)
        enumerated_content = enumerate_file(modification.source_code)

        #Pass to starting point of agent interaction



Load Testcode

In [5]:
import os
code_folder = "codebase"
os.makedirs(code_folder, exist_ok=True)

sample_file_path = os.path.join(code_folder, "example.py")
with open(sample_file_path, "w") as f:
    f.write("""
def calculate_total(items):
    total = 0
    for item in items:
        total += item['price'] * item['quantity']
    return total

# TODO: Add error handling for missing keys
""")

with open(sample_file_path, "r") as f:
    code_content = f.read()

print(code_content)


def calculate_total(items):
    total = 0
    for item in items:
        total += item['price'] * item['quantity']
    return total

# TODO: Add error handling for missing keys



Manually go through each step

In [ ]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy = UserProxyAgent(name="User")

debt_response = user_proxy.initiate_chat(
    recipient=agent1,
    message=f"Analyze the following code and identify technical debt:\n\n{code_content}"
)
print(debt_response)


In [ ]:
response = await ollama_model_client.create([UserMessage(content=f"Analyze the following code and identify technical debt:\n\n{code_content}", source="user")])
print(response)
technical_debt_item = response
await ollama_model_client.close()

In [ ]:
response = await ollama_model_client_suggest.create([UserMessage(content=f"Suggest refactoring for this code,:\n\n{code_content} to fix this technical debt \n\n{technical_debt_item}. Describe how a user should perform the refactoring.", source="user")])
print(response)
suggested_fix = response
await ollama_model_client_suggest.close()

In [ ]:
response = await ollama_model_client_refactor.create([UserMessage(content=f"Refactor this code,:\n\n{code_content} to fix this technical debt item \n\n{technical_debt_item} following this suggestion  \n\n{suggested_fix}. Provide both the description and the full code.", source="user")])
print(response)
refactored_code = response
await ollama_model_client_refactor.close()

Now with agents

In [ ]:
result = await debt_agent.run(task=f"Analyze the following code and identify technical debt:\n\n{code_content}")
print(result)
identified_debt = result

In [ ]:
result = await suggestion_agent.run(task=f"Suggest refactoring for this code,:\n\n{code_content} to fix this technical debt \n\n{identified_debt}. Describe how a user should perform the refactoring.")
print(result)
fix = result

In [ ]:
result = await refactoring_agent.run(task=f"Refactor this code,:\n\n{code_content} to fix this technical debt item \n\n{technical_debt_item} following this suggestion  \n\n{fix}. Provide both the description and the full code.")
print(result)

Now we can create a selector group chat, to allow for more complex interactions between agents.

Here, a selector agent will be created to allow for the code to be analysed until it has passed a test. The selector will make a choice depending on the output of th test agent, and will either pass the refactored code further or make the refactoring agent attempt another refactoring with further information from the test agent.

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import List, Sequence
#from autogen_agentchat import GroupChatManager


text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

model_client = OllamaChatCompletionClient(model="qwen3:0.6b")


selector_prompt = """You are coordinating a multi-agent workflow focused on identifying and resolving technical debt.

Agent roles:
{roles}

Conversation history:
{history}

Based on the above context, select the most appropriate agent from {participants} to perform the next step in the workflow.

Important:
- Ensure that the Technical_debt_identifier has initiated the process before others proceed.
- Follow the logical sequence: identification → suggestion → refactoring → testing → approval.
- Only one agent should be selected at a time.

Choose the agent best positioned to advance the current task.
"""

# Candidate function to narrow down eligible agents#
#def candidate_func(messages: List[BaseChatMessage], agents: List[AssistantAgent]) -> List[AssistantAgent]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    last_msg = messages[-1]
    last_sender = messages[-1].source


    if last_sender == "user":
        return [debt_agent.name]
    if last_sender == "Technical_debt_identifier":
        return [suggestion_agent.name]
    elif last_sender == "Technical_debt_refactoring_suggestor":
        return [refactoring_agent.name]
    elif last_sender == "Technical_debt_refactoring_agent":
        return [test_agent.name]
    elif last_sender == "Test_agent":
        print("-------------------> last msg",last_msg)
        if "approved" in last_msg or "looks good" in last_msg or "yes" in last_msg:
            return []  # Terminate
        else:
            return [refactoring_agent.name]  # Loop back for improvements
    return agents  # Default fallback

def candidate_func_adapter(thread) -> List[AssistantAgent]:
    print(thread)
    return candidate_func(thread.messages, thread.agents)

    
# Create the group chat
team = SelectorGroupChat(
    [debt_agent, suggestion_agent, refactoring_agent, test_agent],
    model_client=model_client,
    #selector=WorkflowSelector(),
    termination_condition=termination,
    candidate_func=candidate_func,
)
# Create the group chat
#group_chat = SelectorGroupChat(
#    agents=[reviewer, fixer, tester],
#    selector=select_speaker,  # Default model-based selector
#    candidate_func=candidate_func,
#    allow_repeat_speaker=False
#)
task = f"Analyze the following code and identify technical debt:\n\n{code_content}. Then suggest a refactoring strategy and refactor the code."
await Console(team.run_stream(task=task))

#manager = GroupChatManager(groupchat=group_chat)

# Entry point
#user_proxy = UserProxyAgent(name="User", code_execution_config=False)

# Start the chat
#user_proxy.initiate_chat(
#    manager,
#    message="Here's a piece of code. Can you help me review and improve it?"
#)

---------- TextMessage (user) ----------
Analyze the following code and identify technical debt:


def calculate_total(items):
    total = 0
    for item in items:
        total += item['price'] * item['quantity']
    return total

# TODO: Add error handling for missing keys
. Then suggest a refactoring strategy and refactor the code.
